In [1]:
import cv2
import PoseModule as pm  # Your poseDetector class

def main():
    cap = cv2.VideoCapture(0)
    detector = pm.poseDetector()
    rep_count = 0
    stage = None  # "open" or "closed"
    feedback = "Get Ready!"

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame. Exiting ...")
            break

        frame = detector.findPose(frame, draw=True)
        lmList = detector.findPosition(frame, draw=False)

        if lmList:
            # Track hands and feet
            left_hand = lmList[11]   # LEFT_SHOULDER
            right_hand = lmList[12]  # RIGHT_SHOULDER
            left_foot = lmList[23]   # LEFT_HIP
            right_foot = lmList[24]  # RIGHT_HIP

            # Check if arms are raised above head
            hands_up = left_hand[1] < lmList[0][1] and right_hand[1] < lmList[0][1]  # y coord comparison with nose
            # Check if legs are apart
            legs_apart = abs(lmList[23][0] - lmList[24][0]) > 0.25  # x coord difference

            # Determine stage
            if hands_up and legs_apart:
                if stage != "open":
                    stage = "open"
                    rep_count += 1
                    feedback = "Good! Keep Going"
            else:
                stage = "closed"
                feedback = "Open Arms & Legs"

            # Display rep count and feedback
            cv2.putText(frame, f"Jumping Jacks: {rep_count}", (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 3)
            cv2.putText(frame, feedback, (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        else:
            feedback = "Step into view"
            cv2.putText(frame, feedback, (10, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        cv2.imshow("Jumping Jack Tracker", frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
